In [1]:
import gym
from gym import wrappers
import torch
import torch.nn as nn
import torch.nn.init
import torch.nn.functional as F
from collections import namedtuple
from torch.autograd import Variable
import random
import numpy as np
from collections import deque

In [2]:
env = gym.make('CartPole-v0')
env = wrappers.Monitor(env, 'cartpole', force=True)

[2017-08-11 15:07:03,507] Making new env: CartPole-v0


In [3]:
class Policy(nn.Module):
  def __init__(self, input_size, output_size):
    super(Policy, self).__init__() # this statement is always needed
    
    self.fc1 = nn.Linear(input_size, 10) # matrix multiplication
    self.fc2 = nn.Linear(10, output_size) # matrix multiplication
    
    # == parameters initialization ==
    nn.init.xavier_normal(self.fc1.weight)
    nn.init.xavier_normal(self.fc2.weight)
    
    nn.init.normal(self.fc1.bias)
    nn.init.normal(self.fc2.bias)
    # =============================== 
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [4]:
class ValueFunction(nn.Module):
  def __init__(self, input_size):
    super(ValueFunction, self).__init__()
    
    self.fc1 = nn.Linear(input_size, 10)
    self.fc2 = nn.Linear(10, 1) # a single value
    
    # == parameters initialization ==
    nn.init.xavier_normal(self.fc1.weight)
    nn.init.xavier_normal(self.fc2.weight)
    
    nn.init.normal(self.fc1.bias)
    nn.init.normal(self.fc2.bias)
    # ===============================
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [5]:
class ActorCritic(nn.Module):
  def __init__(self, input_size, output_size):
    super(ActorCritic, self).__init__()
    self.actor = Policy(input_size, output_size)
    self.critic = ValueFunction(input_size)

  def forward(self, x):
    return self.critic(x), self.actor(x) # value, action

In [6]:
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
print('input_size = {0}, output_size = {1}'.format(input_size, output_size))

input_size = 4, output_size = 2


In [7]:
gamma = 0.99 # the parameter for discounting future rewards
lbda = 0.9 # the parameter for GAE (generalized advantage estimation)

In [8]:
model = ActorCritic(input_size, output_size)
optimizer = torch.optim.Adam(model.parameters())

In [10]:
# vanilla policy gradient
for i in range(2000):
  current_state = env.reset() # an array of 4 values
  done = False
  episode_reward = 0
  
  values = [] 
  logprobs = []
  rewards = []

  while not done:
     # forward propagation on policy using current_state
    value, action_real = model(Variable(torch.from_numpy(current_state).float().unsqueeze(0))) 
    # value and action_real are variables
    action_logprob = F.log_softmax(action_real) # returns a variable
    
    action_prob = F.softmax(action_real) # returns a variable
    action = action_prob.multinomial().data # returns a torch tensor
    
    logprob = action_logprob.gather(1, Variable(action))
    current_state, reward, done, _ = env.step(action.numpy()[0,0])
    
    values.append(value) # variable
    logprobs.append(logprob) # variable
    rewards.append(reward) # numpy
    
    episode_reward += reward
  
  R = 0
  value_loss = 0
  policy_loss = 0
  for j in reversed(range(len(rewards))):
    R = rewards[j] + gamma * R # numpy
    advantage = R - values[j]
    value_loss += advantage.pow(2)
    policy_loss -= logprobs[j] * advantage
  
  optimizer.zero_grad()
  value_loss.backward(retain_variables=True)
  policy_loss.backward()
  optimizer.step()
  
  print('{0}: episode_rewards = {1}, policy_loss = {2}, value_loss = {3}'.format(
    i, episode_reward, policy_loss.data[0,0], value_loss.data[0,0]))

[2017-08-11 15:07:56,226] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.85721.video000001.mp4
[2017-08-11 15:07:56,379] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.85721.video000008.mp4


0: episode_rewards = 9.0, policy_loss = 0.4540056884288788, value_loss = 202.0786590576172
1: episode_rewards = 9.0, policy_loss = 0.5151804089546204, value_loss = 203.80227661132812
2: episode_rewards = 11.0, policy_loss = 0.5992369055747986, value_loss = 357.1293640136719
3: episode_rewards = 9.0, policy_loss = 0.4409686326980591, value_loss = 198.54904174804688
4: episode_rewards = 9.0, policy_loss = 0.48016950488090515, value_loss = 200.05174255371094
5: episode_rewards = 9.0, policy_loss = 0.47633662819862366, value_loss = 197.91897583007812
6: episode_rewards = 8.0, policy_loss = 0.3755933940410614, value_loss = 141.96722412109375
7: episode_rewards = 10.0, policy_loss = 0.4205549955368042, value_loss = 261.80126953125
8: episode_rewards = 9.0, policy_loss = 0.4100188612937927, value_loss = 196.6254119873047
9: episode_rewards = 10.0, policy_loss = 0.42579180002212524, value_loss = 260.97613525390625
10: episode_rewards = 11.0, policy_loss = 0.5124562978744507, value_loss = 345.3

[2017-08-11 15:07:56,606] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.85721.video000027.mp4


12: episode_rewards = 9.0, policy_loss = 0.35967376828193665, value_loss = 189.41232299804688
13: episode_rewards = 10.0, policy_loss = 0.39962002635002136, value_loss = 257.19232177734375
14: episode_rewards = 10.0, policy_loss = 0.38829341530799866, value_loss = 254.92788696289062
15: episode_rewards = 8.0, policy_loss = 0.28707417845726013, value_loss = 134.04171752929688
16: episode_rewards = 9.0, policy_loss = 0.33131369948387146, value_loss = 186.01638793945312
17: episode_rewards = 8.0, policy_loss = 0.31050679087638855, value_loss = 132.65701293945312
18: episode_rewards = 10.0, policy_loss = 0.377653568983078, value_loss = 250.53289794921875
19: episode_rewards = 10.0, policy_loss = 0.37164852023124695, value_loss = 251.1102294921875
20: episode_rewards = 9.0, policy_loss = 0.30034303665161133, value_loss = 183.07298278808594
21: episode_rewards = 10.0, policy_loss = 0.35380735993385315, value_loss = 249.25430297851562
22: episode_rewards = 9.0, policy_loss = 0.294263452291488

[2017-08-11 15:07:56,912] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.85721.video000064.mp4


...

1790: episode_rewards = 200.0, policy_loss = 2861.952392578125, value_loss = 228184.96875
1791: episode_rewards = 200.0, policy_loss = 2796.263671875, value_loss = 201330.46875
1792: episode_rewards = 200.0, policy_loss = 2548.916015625, value_loss = 223180.53125
1793: episode_rewards = 200.0, policy_loss = 2348.494140625, value_loss = 211709.75
1794: episode_rewards = 200.0, policy_loss = 2547.34228515625, value_loss = 218190.203125
1795: episode_rewards = 200.0, policy_loss = 2478.6103515625, value_loss = 209455.03125
1796: episode_rewards = 105.0, policy_loss = 185.89710998535156, value_loss = 29286.986328125
1797: episode_rewards = 200.0, policy_loss = 2224.85888671875, value_loss = 208186.75
1798: episode_rewards = 200.0, policy_loss = 2601.0087890625, value_loss = 206835.046875
1799: episode_rewards = 200.0, policy_loss = 2384.2939453125, value_loss = 206695.15625
1800: episode_rewards = 200.0, policy_loss = 2552.56298828125, value_loss = 208312.375
1801: episode_rewards = 200.0,

1882: episode_rewards = 200.0, policy_loss = 2060.97998046875, value_loss = 169322.578125
1883: episode_rewards = 200.0, policy_loss = 2024.3671875, value_loss = 172820.296875
1884: episode_rewards = 172.0, policy_loss = 1234.513427734375, value_loss = 138708.96875
1885: episode_rewards = 200.0, policy_loss = 2035.096923828125, value_loss = 177234.359375
1886: episode_rewards = 200.0, policy_loss = 2083.822509765625, value_loss = 169474.125
1887: episode_rewards = 200.0, policy_loss = 2190.9970703125, value_loss = 171762.71875
1888: episode_rewards = 200.0, policy_loss = 2069.1796875, value_loss = 162745.6875
1889: episode_rewards = 200.0, policy_loss = 1762.640869140625, value_loss = 179456.90625
1890: episode_rewards = 200.0, policy_loss = 1797.278076171875, value_loss = 183658.265625
1891: episode_rewards = 200.0, policy_loss = 2250.953125, value_loss = 168037.671875
1892: episode_rewards = 200.0, policy_loss = 1950.8798828125, value_loss = 179444.421875
1893: episode_rewards = 200.

1975: episode_rewards = 174.0, policy_loss = 929.8939208984375, value_loss = 117288.015625
1976: episode_rewards = 200.0, policy_loss = 1517.2208251953125, value_loss = 156841.484375
1977: episode_rewards = 200.0, policy_loss = 1627.0682373046875, value_loss = 154239.921875
1978: episode_rewards = 200.0, policy_loss = 1706.16943359375, value_loss = 155471.78125
1979: episode_rewards = 200.0, policy_loss = 1461.419677734375, value_loss = 156414.078125
1980: episode_rewards = 191.0, policy_loss = 1272.302490234375, value_loss = 145794.015625
1981: episode_rewards = 200.0, policy_loss = 1481.617431640625, value_loss = 161632.640625
1982: episode_rewards = 200.0, policy_loss = 1618.44189453125, value_loss = 157397.0
1983: episode_rewards = 200.0, policy_loss = 1647.8310546875, value_loss = 160619.765625
1984: episode_rewards = 200.0, policy_loss = 1584.98828125, value_loss = 157966.125
1985: episode_rewards = 200.0, policy_loss = 1667.3214111328125, value_loss = 156353.734375
1986: episode

[2017-08-11 15:08:55,784] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.85721.video002000.mp4


1998: episode_rewards = 200.0, policy_loss = 1496.383056640625, value_loss = 154714.140625
1999: episode_rewards = 200.0, policy_loss = 1553.1297607421875, value_loss = 151371.015625


In [11]:
env.render(close=True)
env.close()

[2017-08-11 15:08:57,476] Finished writing results. You can upload them to the scoreboard via gym.upload('/Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole')


In [12]:
env = gym.make('Pendulum-v0')

[2017-08-11 15:30:57,078] Making new env: Pendulum-v0


In [21]:
input_size = env.observation_space.shape[0]
output_size = env.action_space.shape[0]
print('input_size = {0}, output_size = {1}'.format(input_size, output_size))

input_size = 3, output_size = 1


In [22]:
model = ActorCritic(input_size, output_size)
optimizer = torch.optim.Adam(model.parameters())

In [23]:
# vanilla policy gradient
for i in range(2000):
  current_state = env.reset() # an array of 4 values
  done = False
  episode_reward = 0
  
  values = []
  logprobs = []
  rewards = []

  while not done:
     # forward propagation on policy using current_state
    value, action_real = model(Variable(torch.from_numpy(current_state).float().unsqueeze(0)))
    break
    # value and action_real are variables
#     action_logprob = F.log_softmax(action_real) # returns a variable
    
#     action_prob = F.softmax(action_real) # returns a variable
#     action = action_prob.multinomial().data # returns a torch tensor
    
#     logprob = action_logprob.gather(1, Variable(action))
#     current_state, reward, done, _ = env.step(action.numpy()[0,0])
    
#     values.append(value) # variable
#     logprobs.append(logprob) # variable
#     rewards.append(reward) # numpy
    
#     episode_reward += reward
  
#   R = 0
#   value_loss = 0
#   policy_loss = 0
#   for j in reversed(range(len(rewards))):
#     R = rewards[j] + gamma * R # numpy
#     advantage = R - values[j]
#     value_loss += advantage.pow(2)
#     policy_loss -= logprobs[j] * advantage
  
#   optimizer.zero_grad()
#   value_loss.backward(retain_variables=True)
#   policy_loss.backward()
#   optimizer.step()
  
#   print('{0}: episode_rewards = {1}, policy_loss = {2}, value_loss = {3}'.format(
#     i, episode_reward, policy_loss.data[0,0], value_loss.data[0,0]))

In [25]:
F.tanh(action_real)

Variable containing:
 0.9017
[torch.FloatTensor of size 1x1]